In [1]:
import pandas as pd
from numpy  import *
import numpy as np
import matplotlib.pyplot as plt

__You need the files saved with names "movements_unique" and "Farms_2010_RSG" in your folder__

In [ ]:
movement_data=pd.read_csv("movements_unique",header=None, delim_whitespace=True)
farmnum_from=movement_data.values[:,0]
farmnum_to=movement_data.values[:,1]
weight_link=movement_data.values[:,2]
farms_2010=pd.read_csv("Farms_2010_RSG",names=["county number", "parish number", "holding number", 
                                               "X coordinate", "Y coordinate", "number of cattle", 
                                               "number of sheep"],delim_whitespace=True)
Cumbparish=pd.DataFrame()
Cumbparish.columns=['parish from','parish to','weight link']


__We add the counties and parishes to the data frame. Note that parishes with the same number exist in different counties__

In [ ]:
county_2010=farms_2010.values[:,0]
county_to=np.zeros(len(farmnum_to))
for i in range(len(farmnum_to)):
    county_to[i]=county_2010[int(farmnum_to[i])-1]
    
county_from=np.zeros(len(farmnum_from))
for i in range(len(farmnum_from)):
    county_from[i]=county_2010[int(farmnum_from[i])-1]
movement_data[3]=county_from
movement_data[4]=county_to

In [ ]:
parish_2010=farms_2010.values[:,1]
parish_to=np.zeros(len(farmnum_to))
for i in range(len(farmnum_to)):
    parish_to[i]=parish_2010[int(farmnum_to[i])-1]
    
parish_from=np.zeros(len(farmnum_from))
for i in range(len(farmnum_from)):
    parish_from[i]=parish_2010[int(farmnum_from[i])-1]
movement_data[5]=parish_from
movement_data[6]=parish_to

__We look at cumbria which is county number 8.__

In [ ]:
cumbData = farms_2010[farms_2010['county number'] == 8]
Cumbonlymov=[]
for i in range(len(county_to)):
    if ((county_to[i]==8) and (county_from[i]==8)):
        Cumbonlymov.append(i)
        
#I don't really need this function here it was for a previous purpose, the line below does this anyway

cumbparishmov=movement_data[(movement_data[3]==8) & (movement_data[4]==8)]
cumbparishmov.columns=['farm number from','farm number to','weight link'
                       ,'county from','county to','parish from','parish to']


In [ ]:
cumbparishmov2=cumbparishmov.drop(['farm number from','farm number to','county from','county to'],axis=1)
cumparwl=cumbparishmov2.values[:,0]
cumparf=cumbparishmov2.values[:,1]
cumpart=cumbparishmov2.values[:,2]

#these are short for cumbria parish from, cumbria parish to etc
#NOTE: 0 is the column which contains weights

__This bit of code adds together movements that go from and to the same parishes. So we get an overall weight for the movement between parishes__

In [ ]:
cumparf2=cumparf[0]
cumpart2=cumpart[0]
cumparwl2=cumparwl[0]
for i in range(len(cumparf)-1):
    a=cumparf[i+1]
    b=cumpart[i+1]
    [stat1]=np.where((cumparf2==a) & (cumpart2==b))
    if stat1.size:
        cumparwl2[cumparf2==cumparf[i+1]]=cumparwl2[cumparf2==cumparf[i+1]]+cumparwl[i+1]
        
    else:
        cumparf2=np.append(cumparf2,cumparf[i+1])
        cumpart2=np.append(cumpart2,cumpart[i+1])
        cumparwl2=np.append(cumparwl2,cumparwl[i+1])

In [ ]:
Cumbparish['parish from']=cumparf2
Cumbparish['parish to']=cumpart2
Cumbparish['weight link']=cumparwl2

__This saves a CSV file__

In [ ]:
Cumbparish.to_csv("Cumbparish.csv",index=False)

In [ ]:
unweightedCumbparish=Cumbparish.drop('weight link',axis=1)

In [ ]:
unweightedCumbparish.to_csv("unweightedCumbparish.csv",index=False)